In [1]:
%%capture
pip install overpy

In [2]:
import pandas as pd

file_path1 = 'competitors_MI.csv'
competitors_MI = pd.read_csv(file_path1)

file_path2 = 'meijer_stores_MI.csv'
meijer_stores_MI = pd.read_csv(file_path2)

In [9]:
meijer_stores_MI.head(4)

,name,address,latitude,longitude,county
0,Meijer,"1251 M-32, Alpena, MI 49707, United States",45.059147,-83.474693,Alcona
1,Meijer,"100 Pigeon Rd, Bad Axe, MI 48413, United States",43.825699,-83.004745,Alcona
2,Meijer,"9515 Birch Run Rd, Birch Run, MI 48415, United...",43.252918,-83.765800,Alcona
3,Meijer,"3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...",43.482030,-83.982479,Alcona
4,Meijer Express Gas Station,"1241 M-32, Alpena, MI 49707, United States",45.061162,-83.474606,Alcona
...,...,...,...,...,...
199,Meijer Express Gas Station,"521 N Pine Rd, Bay City, MI 48708, United States",43.580492,-83.837882,Tuscola
200,Meijer Express Gas Station,"5671 Jackson Rd, Ann Arbor, MI 48103, United S...",42.287728,-83.842059,Washtenaw
201,Meijer Express Gas Station,"3995 Carpenter Rd, Ypsilanti, MI 48197, United...",42.231416,-83.679345,Washtenaw
202,Meijer,"1475 E Jefferson Ave, Detroit, MI 48207, Unite...",42.334396,-83.032600,Wayne


In [4]:
meijer_store_MI_sample = meijer_stores_MI.head(4)

In [5]:
competitors_MI.head(4)

,name,address,latitude,longitude,County
0,Kum & Go,"4800 44th St SW, Grandville, MI 49418, United ...",42.884595,-85.781344,Kent County
1,Kum & Go,"5437 Division Ave S, Wyoming, MI 49548, United...",42.865430,-85.664682,Kent County
2,Kum & Go,"4560 Chicago Dr SW, Grandville, MI 49418, Unit...",42.905083,-85.776313,Kent County
3,Kum & Go,"2134 Alpine Ave NW, Walker, MI 49544, United S...",43.002258,-85.688179,Kent County


In [6]:
competitors_MI_sample = competitors_MI.head(4)

In [8]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# Convert miles to meters for the radius
radius_miles = 1
radius = radius_miles * 1609.34

# Define categories and their tags
categories_tags = {
    
    "Grocery Stores": [
        ("amenity", "supermarket"),
        ("shop", "Bakery"),
        ("shop", "Convenience"),
        ("shop", "Frozen Food"),
        ("shop", "Department Store"),
        ("shop", "Supermarket"),
        ("shop", "Wholesale"),
    ],
    
    "Food Outlets": [
        ("amenity", "Fast Food"),
        ("amenity", "Food Court"),
        ("amenity", "restaurant"),
        ("amenity", "Ice Cream"),
        ("shop", "Cheese"),
        ("shop", "Chocolate"),
        ("shop", "Ice Cream"),
        ("shop", "Pasta"),
        ("shop", "Pastry"),
        ("shop", "Seafood"),
        ("shop", "Food"),
    ],
    
    "Tourist Destinations": [
        ("tourism", "attraction"),
        ("amenity", "Events Venue"),
        ("amenity", "Exhibition Centre"),
        ("boundary", "national_park"),
        ("boundary", "Museum"),
        ("building", "Beach Hut"),
        ("building", "Castle"),
    ],
    
    "Hotels": [
        ("building", "hotel"),
    ],
    
    "Educational Institutions": [
        ("building", "College"),
        ("building", "School"),
        ("amenity", "library"),
        ("building", "University"),
    ],
    
    "Hospitals": [
        ("amenity", "Clinic"),
        ("amenity", "Nursing Home"),
        ("amenity", "Pharmacy"),
        ("building", "hospital"),
    ],
    
    "Airports": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
    ],
    
    "Residential Housing Density": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "Hut"),
    ],
    
    "Commercial Housing Density": [
        ("building", "commercial"),
        ("building", "office"),
    ],
    
    "Fuel Stations & Car Wash": [
        ("amenity", "Car Wash"),
        ("amenity", "Charging Station"),
        ("amenity", "Fuel"),
    ],
    
    "Apparels": [
        ("shop", "Clothes"),
        ("shop", "Fabric"),
        ("shop", "Fashion"),
    ],
    
    "Height Restrictor": [
        ("barrier", "height_restrictor"),
    ],
    
    "Border Zone": [
        ("boundary", "border_zone"),
    ],
    
    "Public Transportation": [
        ("public_transport", "Public Transport Platform"),
        ("public_transport", "Public Transport Station"),
        ("public_transport", "Railway Station"),
        ("public_transport", "Railway Subway Entrance"),
        ("public_transport", "Railway Tram Stop"),
    ],
}


# Define a delay duration in seconds (e.g., 10 seconds)
delay_duration = 5

def fetch_feature_count(api, lat, lon, radius, category_tags):
    """
    Fetch count of features for the given category tags.

    :param api: Overpass API instance.
    :param lat: Latitude of the center.
    :param lon: Longitude of the center.
    :param radius: Radius for the search.
    :param category_tags: Dictionary of categories and their tags.
    :return: Dictionary with counts of features for each category.
    """
    category_counts = {}

    for category, tags in category_tags.items():
        print(f"Fetching data for category: {category}")
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count
        print(f"Completed fetching for category: {category}. Count: {category_count}")
        
        # Delay to manage server load
        print(f"Waiting for {delay_duration} seconds to manage server load.")
        time.sleep(delay_duration)

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        print(f"Processing location at latitude: {lat}, longitude: {lon}")
        feature_counts = fetch_feature_count(api, lat, lon, radius, categories_tags)
        feature_counts.update({
            "latitude": lat,
            "longitude": lon,
            "radius_miles": radius_miles
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    meijer_stores_MI = pd.DataFrame(all_feature_counts)

    # Display the DataFrame
    print(meijer_stores_MI)

    # Output the DataFrame to a CSV file
    meijer_stores_MI.to_csv("meijer_nearby_places_final.csv", index=False)
    print("Data collection complete. CSV file created.")

if __name__ == "__main__":
    # Input DataFrame with 'latitude' and 'longitude' columns
    meijer_nearby_places_final = main(meijer_stores_MI)

Processing location at latitude: 45.0591468, longitude: -83.4746932
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 2
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetchin

Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 45.0611621, longitude: -83.4746063
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 2
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels

Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 10
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 45.7514114, longitude: -87.0866699
Fetching 

Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 7
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Re

Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housi

...

Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 15
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Re

Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 43.0459056, longitude: -82.4579791
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 6
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Com

Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 41.934064, longitude: -85.655071
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Out

Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 118
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for categor

Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 44
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 7
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel 

In [10]:
file_path = 'meijer_nearby_places_final.csv'
nearby_places_final = pd.read_csv(file_path)
nearby_places_final.head()

,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,latitude,longitude,radius_miles
0,0,2,0,0,0,0,0,17,5,0,0,0,0,0,45.059147,-83.474693,1
1,0,3,0,0,0,0,0,0,1,0,0,0,0,0,43.825699,-83.004745,1
2,0,8,0,1,0,0,0,0,2,0,0,0,0,0,43.252918,-83.765800,1
3,0,19,0,0,0,1,0,0,6,0,0,0,0,0,43.482030,-83.982479,1
4,0,2,0,0,0,0,0,16,5,0,0,0,0,0,45.061162,-83.474606,1


In [11]:
final_nearby_places_meijer = pd.merge(meijer_stores_MI, nearby_places_final, on=['latitude','longitude'], how='inner')
final_nearby_places_meijer

,name,address,latitude,longitude,county,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,radius_miles
0,Meijer,"1251 M-32, Alpena, MI 49707, United States",45.059147,-83.474693,Alcona,0,2,0,0,0,0,0,17,5,0,0,0,0,0,1
1,Meijer,"100 Pigeon Rd, Bad Axe, MI 48413, United States",43.825699,-83.004745,Alcona,0,3,0,0,0,0,0,0,1,0,0,0,0,0,1
2,Meijer,"9515 Birch Run Rd, Birch Run, MI 48415, United...",43.252918,-83.765800,Alcona,0,8,0,1,0,0,0,0,2,0,0,0,0,0,1
3,Meijer,"3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...",43.482030,-83.982479,Alcona,0,19,0,0,0,1,0,0,6,0,0,0,0,0,1
4,Meijer Express Gas Station,"1241 M-32, Alpena, MI 49707, United States",45.061162,-83.474606,Alcona,0,2,0,0,0,0,0,16,5,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Meijer Express Gas Station,"521 N Pine Rd, Bay City, MI 48708, United States",43.580492,-83.837882,Tuscola,0,4,0,0,0,0,0,2,0,0,0,0,0,0,1
204,Meijer Express Gas Station,"5671 Jackson Rd, Ann Arbor, MI 48103, United S...",42.287728,-83.842059,Washtenaw,0,6,0,0,0,0,0,5,3,0,0,0,0,0,1
205,Meijer Express Gas Station,"3995 Carpenter Rd, Ypsilanti, MI 48197, United...",42.231416,-83.679345,Washtenaw,0,7,0,0,0,0,0,44,7,0,0,0,0,0,1
206,Meijer,"1475 E Jefferson Ave, Detroit, MI 48207, Unite...",42.334396,-83.032600,Wayne,0,82,4,6,3,0,0,143,125,0,0,0,0,0,1


In [12]:
 final_nearby_places_meijer.to_csv("final_nearby_places_meijer.csv", index=False)

# Competitor Stores

In [13]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# Convert miles to meters for the radius
radius_miles = 1
radius = radius_miles * 1609.34

# Define categories and their tags
categories_tags = {
    
    "Grocery Stores": [
        ("amenity", "supermarket"),
        ("shop", "bakery"),
        ("shop", "convenience"),
        ("shop", "frozen food"),
        ("shop", "department store"),
        ("shop", "supermarket"),
        ("shop", "wholesale")
    ],
    
    "Food Outlets": [
        ("amenity", "fast food"),
        ("amenity", "food court"),
        ("amenity", "restaurant"),
        ("amenity", "ice cream"),
        ("shop", "cheese"),
        ("shop", "chocolate"),
        ("shop", "ice cream"),
        ("shop", "pasta"),
        ("shop", "pastry"),
        ("shop", "seafood"),
        ("shop", "food")
    ],
    
    "Tourist Destinations": [
        ("tourism", "attraction"),
        ("amenity", "Events Venue"),
        ("amenity", "Exhibition Centre"),
        ("boundary", "national_park"),
        ("boundary", "Museum"),
        ("building", "Beach Hut"),
        ("building", "Castle")
    ],
    
    "Hotels": [
        ("building", "hotel")
    ],
    
    "Educational Institutions": [
        ("building", "College"),
        ("building", "School"),
        ("amenity", "library"),
        ("building", "University")
    ],
    
    "Hospitals": [
        ("amenity", "Clinic"),
        ("amenity", "nursing home"),
        ("amenity", "pharmacy"),
        ("building", "hospital"),
    ],
    
    "Airports": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
    ],
    
    "Residential Housing Density": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "hut"),
    ],
    
    "Commercial Housing Density": [
        ("building", "commercial"),
        ("building", "office")
    ],
    
    "Fuel Stations & Car Wash": [
        ("amenity", "car wash"),
        ("amenity", "charging station"),
        ("amenity", "fuel")
    ],
    
    "Apparels": [
        ("shop", "clothes"),
        ("shop", "fabric"),
        ("shop", "fashion")
    ],
    
    "Height Restrictor": [
        ("barrier", "height_restrictor")
    ],
    
    "Border Zone": [
        ("boundary", "border_zone")
    ],
    
    "Public Transportation": [
        ("public_transport", "public transport platform"),
        ("public_transport", "public transport station"),
        ("public_transport", "railway station"),
        ("public_transport", "railway subway entrance"),
        ("public_transport", "railway tram stop")
    ],
}


# Define a delay duration in seconds (e.g., 10 seconds)
delay_duration = 1

def fetch_feature_count(api, lat, lon, radius, category_tags):
    """
    Fetch count of features for the given category tags.

    :param api: Overpass API instance.
    :param lat: Latitude of the center.
    :param lon: Longitude of the center.
    :param radius: Radius for the search.
    :param category_tags: Dictionary of categories and their tags.
    :return: Dictionary with counts of features for each category.
    """
    category_counts = {}

    for category, tags in category_tags.items():
        print(f"Fetching data for category: {category}")
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count
        print(f"Completed fetching for category: {category}. Count: {category_count}")
        
        # Delay to manage server load
        print(f"Waiting for {delay_duration} seconds to manage server load.")
        time.sleep(delay_duration)

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        print(f"Processing location at latitude: {lat}, longitude: {lon}")
        feature_counts = fetch_feature_count(api, lat, lon, radius, categories_tags)
        feature_counts.update({
            "latitude": lat,
            "longitude": lon,
            "radius_miles": radius_miles
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    competitors_MI = pd.DataFrame(all_feature_counts)

    # Display the DataFrame
    print(competitors_MI)

    # Output the DataFrame to a CSV file
    competitors_MI.to_csv("competitor_nearby_places_final.csv", index=False)
    print("Data collection complete. CSV file created.")

if __name__ == "__main__":
    # Input DataFrame with 'latitude' and 'longitude' columns
    competitor_nearby_places_final = main(competitors_MI)

Processing location at latitude: 42.8845946, longitude: -85.7813442
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 5
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetchin

Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 42.8688229, longitude: -85.6257608
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels

Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 26
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 42.7660971, longitude: -85.3165607
Fetching 

Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Re

Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housi

...

Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 22
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category

Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 14
Waiting for 5 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel 

Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 3
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 1
Waiting for 5 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Res

Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
Processing location at latitude: 42.260417, longitude: -83.730209
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 19
Waiting for 5 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 3
Waiting for 5 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Educational Institutions
Comp

Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 5 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 5 seconds to manage server load.
     Grocery Stores  Food Outlets  Tourist Destinations  Hotels  \
0                 0             5                     0       0   
1                 0             2                     0       0   
2                 0             6              

In [14]:
file_path = 'competitor_nearby_places_final.csv'
competitor_nearby_places_final = pd.read_csv(file_path)
competitor_nearby_places_final.head()

,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,latitude,longitude,radius_miles
0,0,5,0,0,0,0,0,12,8,0,0,0,0,0,42.884595,-85.781344,1
1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,42.865430,-85.664682,1
2,0,6,0,1,1,0,0,43,17,0,0,0,0,0,42.905083,-85.776313,1
3,0,3,0,0,0,0,0,7,5,0,0,0,0,0,43.002258,-85.688179,1
4,0,1,0,0,0,0,0,6,0,0,0,0,0,0,42.868823,-85.625761,1


In [15]:
final_competitor_nearby_places = pd.merge(competitors_MI, competitor_nearby_places_final, on=['latitude','longitude'], how='inner')
final_competitor_nearby_places

,name,address,latitude,longitude,County,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,radius_miles
0,Kum & Go,"4800 44th St SW, Grandville, MI 49418, United ...",42.884595,-85.781344,Kent County,0,5,0,0,0,0,0,12,8,0,0,0,0,0,1
1,Kum & Go,"5437 Division Ave S, Wyoming, MI 49548, United...",42.865430,-85.664682,Kent County,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Kum & Go,"4560 Chicago Dr SW, Grandville, MI 49418, Unit...",42.905083,-85.776313,Kent County,0,6,0,1,1,0,0,43,17,0,0,0,0,0,1
3,Kum & Go,"2134 Alpine Ave NW, Walker, MI 49544, United S...",43.002258,-85.688179,Kent County,0,3,0,0,0,0,0,7,5,0,0,0,0,0,1
4,Kum & Go,"5215 Kalamazoo Ave SE, Kentwood, MI 49508, Uni...",42.868823,-85.625761,Kent County,0,1,0,0,0,0,0,6,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,Circle K,"27248 Van Dyke Ave, Warren, MI 48093, United S...",42.493987,-83.027142,Macomb County,0,5,0,0,1,0,0,61,0,0,0,0,0,0,1
281,Circle K,"1420 E Stadium Blvd, Ann Arbor, MI 48104, Unit...",42.260417,-83.730209,Washtenaw County,0,19,3,0,0,0,0,190,47,0,0,0,0,0,1
282,Circle K,"1879 N Lapeer Rd, Lapeer, MI 48446, United States",43.083855,-83.308497,Lapeer County,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
283,Circle K,"12800 Hamilton Ave, Highland Park, MI 48203, U...",42.396613,-83.101893,Wayne County,0,2,0,1,0,0,0,110,23,0,0,0,0,0,1


In [16]:
 final_competitor_nearby_places.to_csv("final_competitor_nearby_places.csv", index=False)

# Final Nearby Places DF

In [23]:
import pandas as pd

# Standardize column names to lowercase for both DataFrames
final_competitor_nearby_places.columns = final_competitor_nearby_places.columns.str.lower()
final_nearby_places_meijer.columns = final_nearby_places_meijer.columns.str.lower()

# Now concatenate the DataFrames
final_nearby_places_df = pd.concat([final_competitor_nearby_places, final_nearby_places_meijer], ignore_index=True)

# List of all columns to edit with the '_1mile' suffix
columns_to_edit = [
    'grocery stores', 'food outlets', 'tourist destinations', 'hotels', 
    'educational institutions', 'hospitals', 'airports', 
    'residential housing density', 'commercial housing density', 
    'fuel stations & car wash', 'apparels', 'height restrictor', 
    'border bone', 'public transportation'
]

# Creating the dictionary for renaming
rename_dict = {col: col + ' 1 mile' for col in columns_to_edit}

# Renaming the columns
final_nearby_places_df.rename(columns=rename_dict, inplace=True)

# Removing the 'radius_miles' column
final_nearby_places_df.drop(columns=['radius_miles'], inplace=True)

final_nearby_places_df.to_csv("final_nearby_places_df.csv", index=False)

final_nearby_places_df

,name,address,latitude,longitude,county,grocery stores 1 mile,food outlets 1 mile,tourist destinations 1 mile,hotels 1 mile,educational institutions 1 mile,hospitals 1 mile,airports 1 mile,residential housing density 1 mile,commercial housing density 1 mile,fuel stations & car wash 1 mile,apparels 1 mile,height restrictor 1 mile,border zone,public transportation 1 mile
0,Kum & Go,"4800 44th St SW, Grandville, MI 49418, United ...",42.884595,-85.781344,Kent County,0,5,0,0,0,0,0,12,8,0,0,0,0,0
1,Kum & Go,"5437 Division Ave S, Wyoming, MI 49548, United...",42.865430,-85.664682,Kent County,0,2,0,0,0,0,0,0,0,0,0,0,0,0
2,Kum & Go,"4560 Chicago Dr SW, Grandville, MI 49418, Unit...",42.905083,-85.776313,Kent County,0,6,0,1,1,0,0,43,17,0,0,0,0,0
3,Kum & Go,"2134 Alpine Ave NW, Walker, MI 49544, United S...",43.002258,-85.688179,Kent County,0,3,0,0,0,0,0,7,5,0,0,0,0,0
4,Kum & Go,"5215 Kalamazoo Ave SE, Kentwood, MI 49508, Uni...",42.868823,-85.625761,Kent County,0,1,0,0,0,0,0,6,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,Meijer Express Gas Station,"521 N Pine Rd, Bay City, MI 48708, United States",43.580492,-83.837882,Tuscola,0,4,0,0,0,0,0,2,0,0,0,0,0,0
489,Meijer Express Gas Station,"5671 Jackson Rd, Ann Arbor, MI 48103, United S...",42.287728,-83.842059,Washtenaw,0,6,0,0,0,0,0,5,3,0,0,0,0,0
490,Meijer Express Gas Station,"3995 Carpenter Rd, Ypsilanti, MI 48197, United...",42.231416,-83.679345,Washtenaw,0,7,0,0,0,0,0,44,7,0,0,0,0,0
491,Meijer,"1475 E Jefferson Ave, Detroit, MI 48207, Unite...",42.334396,-83.032600,Wayne,0,82,4,6,3,0,0,143,125,0,0,0,0,0
